In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
key = os.getenv("GEMINI_KEY")

In [19]:
from IPython.display import Markdown
import textwrap

def to_markdown(text):
  """Formats text into Markdown with proper bullet points and indentation.

  Args:
      text: The text string to be formatted.

  Returns:
      An IPython.display.Markdown object containing the formatted text.
  """
  # Replace bullet point symbol (•) with asterisk (*) for Markdown
  text = text.replace('•', '*')
  # Indent the text for readability
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [2]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [4]:
model = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=key,
                             temperature=0.3,convert_system_message_to_human=True)

In [5]:
pdf_loader = PyPDFLoader(r"c:\Users\jay59\Downloads\e-Notes_PDF_All-Units_09052019113530AM.pdf")

In [6]:
pages = pdf_loader.load_and_split()

invalid pdf header: b'\r\n\r\n\r'
incorrect startxref pointer(1)


In [7]:
print(pages[8].page_content)

2 – Problems, State Space Search & 
Heuristic Search Techniques  
 
 5 
 
Gopi Sanghani, CE Department  | 2180703  – Artificial Intelligence  • To solve a problem, w e must specify the global database , the rules, and the control 
strategy.  
• For the 8 puzzle problem that correspond to three components.  
• These elements are the problem states, moves and goal.  
• In this problem each tile configuration is a state.  
• The set of  all possible configuration in the problem space, consists of 3,62,880  different 
configurations o f the 8 tiles and blank space.  
• For the 8 -puzzle, a straight forward description is a 3X3 array of matrix of numbers. 
Initial global database is this description of the initial problem state. Virtually any kind of 
data structure can be used to describe states.  
• A move transforms one problem state into another state.  
 
Figure 1: Solution of 8 Puzzle problem  
 
• The 8 -puzzle is conveniently interpreted as having the following for moves.


In [8]:
len(pages)

117

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
context = "\n\n".join(str(p.page_content) for p in pages)
texts = text_splitter.split_text(context)

In [11]:
print(texts[0])

1 – What is AI?  
 
 1 
 
Gopi Sanghani , CE Department  | 21 80703  – Artificial Intelligence  Artificial Intelligence  
• Artificial intelligence is the study of how to make computers do things which, at moment 
people do better.  
• Artificial intelligence can be viewed from a variety of perspectives.  
• From the perspective of intelligence, artificial intelligence is making machines "intelligent" -- 
acting as we would expect people to act.  
o The inability to distinguish computer responses from human responses is called the 
Turing test.  
o Intelligence requires knowledge.  
• From a business perspective AI is a set of very powerful tools, and methodologies for using 
those tools to solve business problems.  
• From a programming perspective, AI includes the study of symbolic programming, problem 
solving, and search.  
o Typically AI programs f ocus on symbols rather than numeric processing.  
o Problem solving i.e. to achieve a specific goal . 
o Search - rarely  access a sol

In [12]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=key)

In [15]:
vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})

In [16]:
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True

)

In [29]:
question = "Explain the State space representation with water jug problem."
result = qa_chain({"query": question})
ans = result['result']

In [30]:
to_markdown(ans)

> A State space is the set of all states reachable from the initial state.
> A state space forms a graph in which the nodes are states and the arcs between nodes are actions.
> In state space, a path is a sequence of states connected by a sequence of actions.
> The solution of a problem is part of the graph formed by the state space.
> 
> For the water jug problem, the state space representation is as follows:
> 
> - A state is represented by a tuple (x, y) where x represents the amount of water in the 4-gallon jug and y represents the amount of water in the 3-gallon jug.
> - The initial state is (0, 0).
> - The goal state is (2, n) for any value of n.
> - The operators are:
>     - Fill the 4-gallon jug from the pump.
>     - Fill the 3-gallon jug from the pump.
>     - Empty the 4-gallon jug.
>     - Empty the 3-gallon jug.
>     - Pour water from the 4-gallon jug to the 3-gallon jug.
>     - Pour water from the 3-gallon jug to the 4-gallon jug.